# Visualizing Static Structures with MDAnalysis and NanoVer

In this example, we will visualize LSD bound into 5-HT2B receptor with MDAnalysis (without dynamics). We'll explore setting up custom visualizations that will be synchronized between users in VR.

## Serving the structure as a NanoVer frame

First, we're going to read in the system with MDAnalysis. MDAnalysis has to guess the bonds because there aren't any defined in the PDB file, so it'll take a while.

In [1]:
import MDAnalysis as mda

In [2]:
universe = mda.Universe(
    'serotonine_receptor.pdb',
    vdwradii={'Na': 0, 'Cl': 0},
    to_guess=("bonds",),
)

Next, we set up a NanoVer server that's set up for serving structures, or as we call them, *frames*. 

In [3]:
from nanover.app import NanoverImdApplication, basic_info_string

In [4]:
app_server = NanoverImdApplication.basic_server(port=0, name="my-server") # Let the OS choose a port for us

In [ ]:
basic_info_string(app_server)

Now, let's convert the MDAnalysis universe to a NanoVer frame object and send it! We have a handy method for doing just that.

In [ ]:
from nanover.mdanalysis import mdanalysis_to_frame_data

In [7]:
frame = mdanalysis_to_frame_data(universe)

In [8]:
app_server.frame_publisher.send_frame(0,frame) # send the first frame

Done! If you connect to the server from VR, you'll see something like this:

<img src="images/lsd_ball_and_stick.png" alt="LSD Ball and Stick" style="width: 500px;"/>g)


# Let's make it look good

The ball and stick representation is fine, but we can customize it right here from the notebook. First, we connect a client, which will ask the server to change how things look.

**Note**: In future releases, the following will probably be simplified, and you'll be able to do it from within VR.

In [9]:
from nanover.websocket import NanoverImdClient

In [10]:
client = NanoverImdClient.from_app_server(app_server)

The first thing we'll do is hide the *root* layer, and slowly layer things back up.
The `modify()` context allows us to make a bunch of changes, which will all be sent to everyone when we're done.

In [11]:
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True

Now, let's set up some CPK colours for the main atoms

In [12]:
cpk_colours = {
    'N': 'blue',
    'P': '#dca523',
    'C': '#c0c0c0',
    'O': '#fc1c03',
    'S': '#e9ce16'
}

We define a utility method that converts from an MDAnalysis selection string to an array of integers that define a selection in NanoVer

In [13]:
def generate_mdanalysis_selection(selection: str, universe):
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

Create a selection of anything that's a protein, and ignore hydrogens

In [14]:
protein = client.create_selection("Protein", [])
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H", universe))

Next, create a selection of the LSD (and other ligands)

In [15]:
# Select ligands
ligands = client.create_selection("Ligands", [])
with ligands.modify():
    ligands.set_particles(generate_mdanalysis_selection("not protein and not resname DPPC", universe))

And we happen to know that DPPC refers to the lipids, so we grab that too

In [16]:
lipids = client.create_selection("Lipids", [])
with lipids.modify():
    lipids.set_particles(generate_mdanalysis_selection("resname DPPC", universe))

We define another useful function that generates colour gradients from matplotlib

In [17]:
import matplotlib.cm
def get_matplotlib_gradient(name: str):
    cmap = matplotlib.colormaps.get_cmap(name)
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

Now, we'll render the protein using our tetrahedral spline renderer, coloured with the lovely viridis colour scheme. 
We do this by editting the `renderer` settings, which is just a dictionary of settings.

In [18]:
# Render the protein
with protein.modify():
    protein.renderer = {
            'sequence': 'polypeptide',
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('viridis')
            },
            'render': 'geometric spline'
        }
    protein.interaction_method = 'none'

Alternatively, you can colour by secondary structure:

In [19]:
with protein.modify():
    protein.renderer = 'cartoon'
    protein.interaction_method = 'none'

We'll add the ligands back in with CPK liquorice

In [20]:
with ligands.modify():
    ligands.renderer = {
            'color': {
                'type': 'cpk',
                'scheme': cpk_colours,
            },
            'scale': 0.1,
            'render': 'liquorice'
        }
    ligands.interaction_method = 'none'

And the lipids with a scaled down liquorice

In [21]:
with lipids.modify():
    lipids.renderer = {
            'color': {
                'type': 'cpk',
                'scheme': cpk_colours,
            },
            'scale': 0.01,
            'render': 'liquorice'
        }
    lipids.interaction_method = 'none'

You may find the lipids are a bit much - you can easily hide them. Just uncomment and run these lines:

In [22]:
# with lipids.modify():
#     lipids.hide = True

## Visualization based on distance

Finally, we'll render the sidechains near ligand

In [23]:
nearby = client.create_selection("Nearby", [])

In [24]:
with nearby.modify():
    nearby.set_particles(generate_mdanalysis_selection("(protein and (not backbone or name CA)) and same resid as around 4 resname 7LD", universe))

In [25]:
with nearby.modify():
    nearby.renderer = {
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('viridis')
            },
            'scale': 0.05,
            'render': 'cycles'
        }
    nearby.interaction_method = 'none'

In [26]:
with nearby.modify():
    nearby.renderer = {
            'color': {
                'type': 'cpk',
                'scheme': cpk_colours,
            },
            'scale': 0.05,
            'render': 'liquorice'
        }
    nearby.interaction_method = 'none'

Depending on which cells you ran, you'll have something that looks like this, much better! (maybe without the lipids)

<img src="images/lsd_nanover.png" style="width: 400px;  display: inline-block; vertical-align: top;">
<img src="images/lsd_nanover_option_2.png" style="width: 305px;  display: inline-block; vertical-align: top;">

# Tidying Up

In [27]:
client.clear_selections()

In [28]:
client.close()

In [29]:
app_server.close()

# Next Steps

* Build your own [VR trajectory viewer](./mdanalysis_trajectory.ipynb) with MDAnalysis and NanoVer
* Dig into how [frames](../fundamentals/frame.ipynb) are constructed under the hood